In [1]:
import sys

import time_module
import time_module.compute as compute
import time_module.network as network


In [5]:
compute.gemm_time("A100-SXM-80GB", 1024, 1024, 1024, dtype="half")
compute.gemm_time("H100-SXM-80GB", 1024, 1024, 1024, dtype="half")

np.float64(13.0)

In [7]:
qwen3_config = """{
  "architectures": [
    "Qwen3MoeForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "decoder_sparse_step": 1,
  "eos_token_id": 151645,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 12288,
  "max_position_embeddings": 40960,
  "max_window_layers": 94,
  "mlp_only_layers": [],
  "model_type": "qwen3_moe",
  "moe_intermediate_size": 1536,
  "norm_topk_prob": true,
  "num_attention_heads": 64,
  "num_experts": 128,
  "num_experts_per_tok": 8,
  "num_hidden_layers": 94,
  "num_key_value_heads": 4,
  "output_router_logits": false,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000.0,
  "router_aux_loss_coef": 0.001,
  "sliding_window": null,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.51.0",
  "use_cache": true,
  "use_sliding_window": false,
  "vocab_size": 151936
}"""

In [9]:
# Qwen3 235B A22B
expert_gate_proj = (2 * 1536, 4096)
expert_up_proj = (1536, 4096)
expert_down_proj = (4096, 1536)
active_expert = 8
# post_attn_layernorm = (4096, )
q_proj = (8192, 4096)
k_proj = (512, 4096)
v_proj = (512, 4096)
o_proj = (4096, 8192)

# q_norm = (128, ) # by head
# k_norm = (128, ) # by head

post_mlp_layernorm = (8192, )


In [33]:
K = 1024
# ctx_len = 16 * K
# ctx_len = 32 * K
ctx_len = 64 * K
gpu = "H100-SXM-80GB"
tp = 1
cp = 1
ep = 1

total_mlp_time = sum([
    compute.gemm_time(gpu, ctx_len // cp, q_proj[1], q_proj[0] // tp, dtype="half"),
    compute.gemm_time(gpu, ctx_len // cp, k_proj[1], k_proj[0] // tp, dtype="half"),
    compute.gemm_time(gpu, ctx_len // cp, v_proj[1], v_proj[0] // tp, dtype="half"),
    compute.gemm_time(gpu, ctx_len // cp, o_proj[1] // tp, o_proj[0], dtype="half"),
    compute.gemm_time(gpu, ctx_len // ep, expert_gate_proj[1], expert_gate_proj[0], dtype="half"),
    active_expert * compute.gemm_time(gpu, ctx_len // ep, expert_up_proj[0], expert_up_proj[1] // tp, dtype="half"),
    active_expert * compute.gemm_time(gpu, ctx_len // ep, expert_down_proj[0] // tp, expert_down_proj[1], dtype="half")
]) / 1000


total_mlp_time

np.float64(36.395)